In [1]:
from cltk.tokenize.word import WordTokenizer
from cltk.stop.latin.stops import STOPS_LIST
from cltk.stem.latin.j_v import JVReplacer
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.sentence import TokenizeSentence
from collections import defaultdict
import re
import string

import os

tok = TokenizeSentence('latin').tokenize
stop = STOPS_LIST
wt = WordTokenizer('latin')
j = JVReplacer()
lemmatizer = LemmaReplacer('latin')

f = open('./latin_text_latin_library/albertofaix/hist1.txt')

text = f.read()

t = text.splitlines()

In [30]:
t[-1]

'\t The Classics Page'

In [9]:
f.close()

In [23]:
text = text[578:]

In [46]:
text = text.replace('\n',' ')
text = j.replace(text.lower())
text = re.sub(r'[0-9]', '', text)
sent_tokens = tok(text.strip())

In [47]:
sent_tokens = sent_tokens[:-1]

In [146]:
class Stupid_Back_Off:
    
    
    def __init__(self,sentence_iterable):
        
        self.bi_gram = defaultdict(lambda: [0,defaultdict(int)])
        self.uni_gram = defaultdict(lambda: [0,defaultdict(int)])
        self.no_gram = [0,defaultdict(int)]
        self.grams = [self.no_gram,self.uni_gram,self.bi_gram]
        
        for sentence in sentence_iterable:
            sent = ['**','**']+sentence+['**STOP**']
            self._update_bigram(sent)
            self._update_unigram(sent)
            self._update_nogram(sent)
            
 
    def _update_nogram(self,sent):
        for i in range(len(sent)):
            self.no_gram[1][(sent[i])] += 1
            self.no_gram[0] += 1

    def _update_unigram(self,sent):
        for i in range(len(sent)-1):
            self.uni_gram[(sent[i])][1][sent[i+1]] += 1
            self.uni_gram[(sent[i])][0] += 1
            
    def _update_bigram(self,sent):
        for i in range(len(sent)-2):
            self.bi_gram[(sent[i],sent[i+1])][1][sent[i+2]] += 1
            self.bi_gram[(sent[i],sent[i+1])][0] += 1
            
            
    def word_proba(self,word,prev=(None),n=2):
        if n == 1 or n == 2:
            return self.grams[n][prev][1][word] / self.grams[n][prev][0]
        else:
            return self.grams[0][1][word] / self.grams[0][0]
        
    def S(self,word,prev_words,i=2):
        """
        Stupid Back-Off model for trigrams.
        P(word | prev_words)
        """
        if i == 0:
            return self.grams[0][1][word] / self.grams[0][0]

        if self.grams[i][prev_words][1][word] > 0:
            return self.grams[i][prev_words][1][word] / self.grams[i][prev_words][0]
        else:
            return .4*self.S(word,prev_words[1:],i-1)

In [144]:
sentences = []

In [145]:
for sent in sent_tokens:
    sentences.append(wt.tokenize(re.sub(r'[{}]'.format(string.punctuation),'',sent)))

In [147]:
sb = Stupid_Back_Off(sentences)

In [153]:
sb.S('quis',('**','si'))

0.00021057521096229777

In [83]:
sent_tokens[2]

'non cinnae, non sullae longa dominatio; et pompei crassique potentia cito in caesarem, lepidi atque antonii arma in augustum cessere, qui cuncta discordiis ciuilibus fessa nomine principis sub imperium accepit.'